In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import gc
from sklearn.metrics import log_loss
import numpy as np
import pickle 

            

In [2]:
train_index = pickle.load(open(FILE.train_index.value,'rb'))
print(len(train_index))
holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))
print(len(holdout_index))

train = pd.read_pickle(FILE.train_base_line.value)
print('train shape is: {}'.format(train.shape))
# test = pd.read_pickle(FILE.test_agg_v1.value)
test = pd.read_pickle(FILE.test_base_line.value)
print('test shape is: {}'.format(test.shape))

961626
40024
train shape is: (1001650, 33)
test shape is: (40024, 32)


In [3]:
base_line_split = pd.read_csv('../../data/features/feature_importance_split_baseline.csv')
base_line_gain = pd.read_csv('../../data/features/feature_importance_gain_baseline.csv')
NUM = 5
categorical_cols = list(set(list(base_line_split.feature.values[:NUM]) + list(base_line_gain.feature.values[:NUM])))
categorical_cols

['creative_type', 'app_id', 'city', 'advert_name', 'osv', 'user_tags', 'model']

In [4]:
def get_new_name(cob_col,func):
    return '+'.join(cob_col)+'_'+func.__name__

In [5]:
func = target_mean
saving_path_mean_train = FILE.train_agg_v3_mean.value
saving_path_mean_test = FILE.test_agg_v3_mean.value


try:
    train_mean = pd.read_pickle(saving_path_mean_train)
    test_mean = pd.read_pickle(saving_path_mean_test)
except:
    train_mean = pd.DataFrame()
    test_mean = pd.DataFrame()
counter = 0
for n_comb in range(1,len(categorical_cols)+1):
    for col in combinations(categorical_cols,n_comb):
        print('#######   {}/{}   ########'.format(col,counter))
        col = list(col)
        counter+=1
        feature_name = get_new_name(col,func)
        if feature_name in train_mean.columns and feature_name in test_mean.columns:
            continue
        train_mean[feature_name],test_mean[feature_name] = func(train,test,train_index,holdout_index,col=col)
        if 'instance_id' not in train_mean.columns:
            train_mean['instance_id'] = train['instance_id'].values
        if 'instance_id' not in test_mean.columns:
            test_mean['instance_id'] = test['instance_id'].values
        train_mean.to_pickle(saving_path_mean_train)
        print(train_mean.shape)
        test_mean.to_pickle(saving_path_mean_test)
        print(test_mean.shape)
    
    

#######   ('creative_type',)/0   ########
(1001650, 5)
(40024, 5)
#######   ('app_id',)/1   ########
(1001650, 6)
(40024, 6)
#######   ('city',)/2   ########
#######   ('advert_name',)/3   ########
#######   ('osv',)/4   ########
(1001650, 7)
(40024, 7)
#######   ('user_tags',)/5   ########
(1001650, 8)
(40024, 8)
#######   ('model',)/6   ########
#######   ('creative_type', 'app_id')/7   ########
(1001650, 9)
(40024, 9)
#######   ('creative_type', 'city')/8   ########
(1001650, 10)
(40024, 10)
#######   ('creative_type', 'advert_name')/9   ########
(1001650, 11)
(40024, 11)
#######   ('creative_type', 'osv')/10   ########
(1001650, 12)
(40024, 12)
#######   ('creative_type', 'user_tags')/11   ########
(1001650, 13)
(40024, 13)
#######   ('creative_type', 'model')/12   ########
(1001650, 14)
(40024, 14)
#######   ('app_id', 'city')/13   ########
(1001650, 15)
(40024, 15)
#######   ('app_id', 'advert_name')/14   ########
(1001650, 16)
(40024, 16)
#######   ('app_id', 'osv')/15   #######

(1001650, 99)
(40024, 99)
#######   ('creative_type', 'app_id', 'city', 'advert_name', 'osv')/98   ########
(1001650, 100)
(40024, 100)
#######   ('creative_type', 'app_id', 'city', 'advert_name', 'user_tags')/99   ########
(1001650, 101)
(40024, 101)
#######   ('creative_type', 'app_id', 'city', 'advert_name', 'model')/100   ########
(1001650, 102)
(40024, 102)
#######   ('creative_type', 'app_id', 'city', 'osv', 'user_tags')/101   ########
(1001650, 103)
(40024, 103)
#######   ('creative_type', 'app_id', 'city', 'osv', 'model')/102   ########
(1001650, 104)
(40024, 104)
#######   ('creative_type', 'app_id', 'city', 'user_tags', 'model')/103   ########
(1001650, 105)
(40024, 105)
#######   ('creative_type', 'app_id', 'advert_name', 'osv', 'user_tags')/104   ########
(1001650, 106)
(40024, 106)
#######   ('creative_type', 'app_id', 'advert_name', 'osv', 'model')/105   ########
(1001650, 107)
(40024, 107)
#######   ('creative_type', 'app_id', 'advert_name', 'user_tags', 'model')/106   #

In [6]:
train = train.merge(train_mean,how='inner',on='instance_id')
print(train.shape)
train.to_pickle(saving_path_mean_train)
        
test = test.merge(test_mean,how='inner',on='instance_id')
print(test.shape)
test.to_pickle(saving_path_mean_test)

(1001650, 160)
(40024, 159)


In [16]:
from os import walk
fileName = []
for (_, _, f) in walk('../../data/Blend/'):
    fileName.extend(f)
    break

In [18]:
def best_blend_param_search(N_weight=5,N_search=10000):

    holdout_y = pickle.load(open('../../data/nn/label/label_holdout.pkl','rb'))

    holdout_list = []
    score_list = []
    param_list = []
    for each in report.fileName.values[:]:
        holdout_list.append(pickle.load(open('../../data/nn/holdout/{}'.format(each),'rb'))['predicted_score'].values)
    holdout_list = np.array(holdout_list)
    for i in tqdm(range(N_search)):
        param = np.random.randint(N_weight,size=(len(holdout_list)))
        preds = np.zeros(holdout_list.shape[1])
        w_sum = 0
        for index in range(len(param)):
            preds += param[index]*holdout_list[index]
            w_sum += param[index]
        if w_sum == 0:
            continue
        preds = preds/w_sum
        score_list.append(log_loss(holdout_y,preds))
        param_list.append(param)
    score_seri = pd.Series(score_list)
    score_seri = score_seri.sort_values()    
    index = score_seri.index.values[0]
    best_score = score_seri.values[0]
    best_param = param_list[index]
    return_df = pd.DataFrame({'fileName':report.fileName.values,
                              'weights':best_param})
    return return_df,best_score

['ho_logistic.csv',
 'wei_4259.csv',
 'wei_4262.csv',
 'nn_blend_0.42607299.csv',
 'wei_4260.csv',
 'shiyi_4258.csv']